In [ ]:
# !pip install torch==2.5.0
# !pip install numpy==1.26.4
# !pip install openai==1.79.0
# !pip install tenacity==9.1.2
# !pip install tiktoken==0.9.0
# !pip install transformers==4.51.3
# !pip install pandas==2.2.3
# !pip install scikit-learn==1.6.1
# !pip install bitsandbytes==0.45.5
# !pip install datasets==3.6.0
# !pip install sentencepiece==0.2.0
# !pip install peft==0.15.2
# !pip install evaluate==0.4.3
# !pip install trl==0.11.4
# !pip install protobuf==6.31.0
# !pip install python-dotenv==1.1.0
# !pip install pandas_ta
# !pip install ollama==0.4.8
# !pip install accelerate==1.7.0
# !pip install ipywidgets
# !pip install pynvml==8.1.7
# !pip uninstall torch torchvision torchaudio -y
# !pip install torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu124

In [ ]:
import pkg_resources

libs = [
    "numpy", "openai", "tenacity", "tiktoken", "transformers", "pandas",
    "scikit-learn", "torch", "bitsandbytes", "datasets", "sentencepiece",
    "peft", "evaluate", "trl", "protobuf", "python-dotenv", "pandas_ta",
    "ollama", "accelerate", "ipywidgets"
]

for lib in libs:
    try:
        version = pkg_resources.get_distribution(lib).version
        print(f"{lib}=={version}")
    except pkg_resources.DistributionNotFound:
        print(f"{lib} not installed")

In [ ]:
import torch # type: ignore
print(torch.cuda.is_available())  # Nếu trả về False, CUDA chưa hoạt động
print(torch.cuda.device_count())  # Kiểm tra số lượng GPU
print(torch.cuda.get_device_name(0))  # Hiển thị tên GPU
# print(torch.set_default_device())

In [23]:
from datasets import load_dataset, Dataset
import re


data_path = "./datasets/"

ds_dict = load_dataset(data_path)

ds_dict

# Lấy tập train từ DatasetDict
ds = ds_dict["train"]


# Tách completion_a và completion_b thành từng dòng mới, rồi tách "Explanation:"
def split_completion(completion_text):
    if "Explanation:" in completion_text:
        parts = completion_text.split("Explanation:", 1)
        target = parts[0].strip()
        explain = "Explanation: " + parts[1].strip()

        explain = explain.replace("\n", "")

    else:
        target = completion_text.strip()
        explain = ""
    return target, explain


# Bước 2: Chuyển đổi từng dòng thành 2 dòng mới
new_data = []
for example in ds:
    prediction_a, explain_a = split_completion(example["completion_a"])
    new_data.append({"user_input": example["user_input"], "completion": example["completion_a"], "prediction": prediction_a, "explain": explain_a})

    prediction_b, explain_b = split_completion(example["completion_b"])
    new_data.append({"user_input": example["user_input"], "completion": example["completion_b"], "prediction": prediction_b, "explain": explain_b})

# Bước 3: Tạo dataset mới từ danh sách đã chuyển đổi
new_ds = Dataset.from_list(new_data)

# Bước 4: In thử dữ liệu
print(new_ds[1]['explain'])
new_ds[1]['explain']
new_ds

Explanation: Despite negative news such as the ban on N95 and surgical masks for the general public and criticisms of being a sweatshop, Amazon's fulfillment of 80,000 out of 100,000 jobs to meet increased demand during the pandemic and Jeff Bezos' donation to Feeding America showcase their commitment to community support. Partnering with SXSW for an online festival and planning to provide masks and temperature checks for staff demonstrate adaptability. With news of Canada's agreement with Amazon Canada and BMO Capital Markets' "Outperform" rating, there are positive signals. Speculations about Amazon's Prime Day event postponement may have led to short-term stock fluctuations, but the overall sentiment remains optimistic.From a technical perspective, despite the daily stock price range and fluctuation, key indicators such as the ADX and RSI suggest a strong bullish momentum. The contact with coronavirus test makers and anticipation for advanced technological solutions like driverless 

Dataset({
    features: ['user_input', 'completion', 'prediction', 'explain'],
    num_rows: 6
})

TypeError: string indices must be integers, not 'str'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForCausalLM, AutoTokenizer # Ví dụ cho LLMs
# from trl import PPOConfig, PPOTrainer # Ví dụ cho PPO
# Các thư viện khác cho ODE solver, flow matching có thể cần thiết

# --- Hyperparameters and Configurations ---
LLM_EXPLANATION_MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf" # Ví dụ
LLM_GUIDANCE_MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"   # Ví dụ
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE_EXPLANATION_LLM = 2e-5
LEARNING_RATE_FLOW_MODEL = 2e-4
NUM_EPOCHS_FLOW_MODEL = 100
NUM_EPOCHS_EXPLANATION_LLM = 10
# ... (Thêm các hyperparameters khác)

# --- 0. Data Loading and Preprocessing ---
def load_dataset(task_name):
    # Input: task_name (e.g., "smac", "mmlu_law")
    # Output: list of (context, actual_decision, all_possible_actions)
    # Ví dụ:
    # if task_name == "smac":
    #     data = [...] # Tải dữ liệu quỹ đạo SMAC
    # elif task_name == "mmlu_law":
    #     data = [...] # Tải dữ liệu MMLU Luật
    # return data
    pass

# --- 1. LLM Components ---
class ExplanationLLM:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(DEVICE)
        # Có thể thêm LoRA adapter ở đây nếu dùng
        self.optimizer = optim.AdamW(self.model.parameters(), lr=LEARNING_RATE_EXPLANATION_LLM)

    def generate_explanation(self, context_text):
        # Input: context_text (string)
        # Output: explanation_sentences (list of strings)
        prompt = f"Given [Context: {context_text}]. Please analyze reasoning for the agent decision based on the context."
        inputs = self.tokenizer(prompt, return_tensors="pt").to(DEVICE)
        # Chú ý: Cần logic để chia output thành các câu
        # outputs = self.model.generate(**inputs, max_new_tokens=200, ...)
        # explanation_full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        # explanation_sentences = explanation_full_text.split('.') # Ví dụ đơn giản
        # return [s.strip() + "." for s in explanation_sentences if s.strip()]
        pass

    def train_step_ppo(self, contexts, explanations, rewards):
        # Input:
        #   contexts: list of context strings
        #   explanations: list of generated explanation (list of sentences)
        #   rewards: list of per-sentence rewards (list of floats)
        # Logic huấn luyện PPO sử dụng TRL hoặc triển khai tùy chỉnh
        # ppo_trainer.step(query_tensors, response_tensors, reward_tensors)
        pass

class GuidanceLLM:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name).to(DEVICE)
        # LLM Hướng dẫn thường được giữ cố định (frozen) hoặc chỉ tinh chỉnh nhẹ
        for param in self.model.parameters():
            param.requires_grad = False
        self.model.eval()

    def get_decision_distribution(self, context_text, explanation_sentences_so_far, all_possible_actions):
        # Input:
        #   context_text: string
        #   explanation_sentences_so_far: list of strings (e.g., s_1, or [s_1, s_2])
        #   all_possible_actions: list of strings representing decisions
        # Output: probability_distribution (torch.Tensor, shape: [num_actions])
        current_explanation = " ".join(explanation_sentences_so_far)
        probs = []
        with torch.no_grad():
            for action in all_possible_actions:
                prompt = f"Given [Context: {context_text}], the reasoning is [{current_explanation}]. Thus, the decision is [{action}]."
                inputs = self.tokenizer(prompt, return_tensors="pt").to(DEVICE)
                # Lấy logits cho token cuối cùng hoặc trung bình logits của token mô tả action
                # outputs = self.model(**inputs)
                # action_logits = ... # Cần logic phức tạp để lấy logit cho toàn bộ action
                # probs.append(action_logits) # Đây sẽ là logit, không phải prob
        # logits_tensor = torch.tensor(probs)
        # probability_distribution = torch.softmax(logits_tensor, dim=0)
        # return probability_distribution
        pass

    def get_last_layer_hidden_states(self, context_text, explanation_sentences_so_far, all_possible_actions_text_for_prompt):
        # Input: như trên, nhưng all_possible_actions_text_for_prompt là một phần của prompt
        # Output: last_hidden_states (torch.Tensor) từ lớp cuối của LLM Hướng dẫn
        # Cần để `output_hidden_states=True` khi gọi model
        # prompt = f"Given [Context: {context_text}], the reasoning is [{current_explanation}]. Thus, the decision is {all_possible_actions_text_for_prompt}."
        # inputs = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
        # with torch.no_grad():
        #     outputs = self.model(**inputs, output_hidden_states=True)
        # last_hidden_states = outputs.hidden_states[-1] # Lấy hidden states của lớp cuối cùng
        # return last_hidden_states
        pass

# --- 2. Rectified Flow Model (φ) ---
class RectifiedFlowModel(nn.Module):
    def __init__(self, guidance_llm_model, num_actions, hidden_dim_guidance_llm, flow_embed_dim=256, projector_layers=4):
        super().__init__()
        self.guidance_llm_model = guidance_llm_model # Tham chiếu đến mô hình LLM Hướng dẫn (đã load)
        self.num_actions = num_actions
        self.hidden_dim_guidance_llm = hidden_dim_guidance_llm # Ví dụ: 4096 cho Llama 7B
        self.flow_embed_dim = flow_embed_dim

        # Embedding cho z_t và PE(t)
        self.zt_embed = nn.Sequential(nn.Linear(num_actions, flow_embed_dim), nn.ReLU(), nn.LayerNorm(flow_embed_dim))
        self.time_embed = nn.Sequential(nn.Linear(1, flow_embed_dim), nn.ReLU(), nn.LayerNorm(flow_embed_dim))

        # Lớp chú ý chéo (Cross-Attention)
        # Trọng số W_Q, W_K, W_V thường được lấy/chia sẻ từ lớp cuối của guidance_llm_model
        # Đây là phần phức tạp nhất để triển khai đúng.
        # self.cross_attention = nn.MultiheadAttention(embed_dim=flow_embed_dim, num_heads=8, kdim=hidden_dim_guidance_llm, vdim=hidden_dim_guidance_llm, batch_first=True)
        # Hoặc một triển khai tùy chỉnh để khớp với mô tả trong bài báo:
        # Query: từ flow_tokens
        # Key/Value: từ flow_tokens + guidance_llm_hidden_states

        # Bộ chiếu (Projector)
        projector_modules = []
        # Input_dim cho projector sẽ là flow_embed_dim (từ h_ATTN_zt) + num_actions (z_t) + 1 (time t) cho skip connections
        # current_dim = flow_embed_dim + num_actions + 1
        # for _ in range(projector_layers - 1):
        #     projector_modules.extend([nn.Linear(current_dim, flow_embed_dim), nn.ReLU(), nn.LayerNorm(flow_embed_dim)])
        #     current_dim = flow_embed_dim + num_actions + 1 # Thêm skip connections vào mỗi lớp
        # projector_modules.append(nn.Linear(current_dim, num_actions))
        # self.projector = nn.Sequential(*projector_modules)
        self.projector = nn.Linear(flow_embed_dim, num_actions) # Phiên bản đơn giản hóa

        self.optimizer = optim.AdamW(self.parameters(), lr=LEARNING_RATE_FLOW_MODEL)

    def forward(self, z_t, time_t, guidance_llm_last_hidden_states):
        # Input:
        #   z_t: (batch_size, num_actions) - trạng thái dòng hiện tại
        #   time_t: (batch_size, 1) - thời gian ODE
        #   guidance_llm_last_hidden_states: (batch_size, seq_len_guidance, hidden_dim_guidance_llm)
        # Output: vector_field (batch_size, num_actions) - tức là φ(t, z_t)

        h_emb_zt = self.zt_embed(z_t)  # (batch_size, flow_embed_dim)
        h_emb_t = self.time_embed(time_t) # (batch_size, flow_embed_dim)

        # Flow tokens: (batch_size, 2, flow_embed_dim)
        flow_tokens_emb = torch.stack([h_emb_zt, h_emb_t], dim=1)

        # --- Cross-Attention Logic ---
        # Đây là phần cốt lõi và phức tạp
        # Giả sử guidance_llm_last_hidden_states đã được chuẩn bị
        # queries_flow = flow_tokens_emb # (B, 2, D_flow)
        # keys_guidance = guidance_llm_last_hidden_states # (B, S_guidance, D_guidance)
        # values_guidance = guidance_llm_last_hidden_states # (B, S_guidance, D_guidance)

        # Lấy trọng số W_Q, W_K, W_V từ LLM Hướng dẫn (cần truy cập vào các tham số của nó)
        # W_q_guidance = self.guidance_llm_model.model.layers[-1].self_attn.q_proj.weight
        # W_k_guidance = self.guidance_llm_model.model.layers[-1].self_attn.k_proj.weight
        # W_v_guidance = self.guidance_llm_model.model.layers[-1].self_attn.v_proj.weight

        # Áp dụng W_Q cho flow_tokens
        # q_flow_projected = torch.matmul(queries_flow, W_q_guidance.T[:self.flow_embed_dim, :self.flow_embed_dim]) # Cần điều chỉnh chiều

        # Áp dụng W_K, W_V cho cả flow_tokens và guidance_llm_last_hidden_states rồi concat
        # ... (logic phức tạp để tạo key và value cho cross-attention)

        # attn_output, _ = self.cross_attention(q_flow_projected, combined_keys, combined_values)
        # h_attn_zt = attn_output[:, 0, :] # Lấy output tương ứng với h_emb_zt

        # Phiên bản đơn giản hóa: bỏ qua cross-attention phức tạp, chỉ dùng h_emb_zt
        h_attn_zt = h_emb_zt # <<<< ĐƠN GIẢN HÓA ĐỂ MINH HỌA

        # --- Projector ---
        # Thêm skip connections (phiên bản đơn giản)
        # projector_input = torch.cat([h_attn_zt, z_t, time_t], dim=1)
        # vector_field = self.projector(projector_input)
        vector_field = self.projector(h_attn_zt) # <<<< ĐƠN GIẢN HÓA

        return vector_field

    def train_step(self, z0_batch, z1_batch, time_batch, guidance_llm_hidden_states_batch):
        # Input:
        #   z0_batch: (batch_size, num_actions) - nhiễu Gaussian
        #   z1_batch: (batch_size, num_actions) - phân phối mục tiêu từ mẫu dương
        #   time_batch: (batch_size, 1) - thời gian t ngẫu nhiên trong [0,1]
        #   guidance_llm_hidden_states_batch: (batch_size, seq_len, hidden_dim) - từ LLM Hướng dẫn
        self.optimizer.zero_grad()
        z_t_batch = time_batch * z1_batch + (1.0 - time_batch) * z0_batch # Nội suy tuyến tính
        target_vector_field = z1_batch - z0_batch
        predicted_vector_field = self.forward(z_t_batch, time_batch, guidance_llm_hidden_states_batch)
        loss = nn.MSELoss()(predicted_vector_field, target_vector_field)
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def solve_ode_generate_distribution(self, z0, guidance_llm_last_hidden_states, num_steps=10):
        # Input:
        #   z0: (num_actions) - điểm bắt đầu nhiễu
        #   guidance_llm_last_hidden_states: (seq_len, hidden_dim)
        #   num_steps: số bước để giải ODE
        # Output: z1_hat (num_actions) - phân phối quyết định ước tính
        # Sử dụng Euler method hoặc ODE solver tốt hơn
        # z_current = z0.unsqueeze(0) # Thêm batch_dim
        # dt = 1.0 / num_steps
        # for i in range(num_steps):
        #     time_t = torch.full((1,1), i * dt, device=DEVICE)
        #     with torch.no_grad():
        #         vector_field = self.forward(z_current, time_t, guidance_llm_last_hidden_states.unsqueeze(0))
        #     z_current = z_current + vector_field * dt
        # return torch.softmax(z_current.squeeze(0), dim=0) # Trả về dạng xác suất
        pass

# --- 3. Main Training Loop ---
def main_training_loop(dataset, num_rounds=2):
    # Khởi tạo các mô hình
    explanation_llm = ExplanationLLM(LLM_EXPLANATION_MODEL_NAME)
    guidance_llm = GuidanceLLM(LLM_GUIDANCE_MODEL_NAME) # Dùng để lấy hidden states và tạo mẫu dương ban đầu

    # Lấy thông tin cần thiết từ guidance_llm cho rectified_flow_model
    # dummy_input_ids = guidance_llm.tokenizer("test", return_tensors="pt")["input_ids"].to(DEVICE)
    # with torch.no_grad():
    #     dummy_outputs = guidance_llm.model(dummy_input_ids, output_hidden_states=True)
    # hidden_dim_guidance = dummy_outputs.hidden_states[-1].shape[-1]
    # num_actions_example = len(dataset[0][2]) # Lấy số action từ mẫu đầu tiên

    # rectified_flow_model = RectifiedFlowModel(
    #     guidance_llm_model=guidance_llm.model, # Truyền tham chiếu đến mô hình đã load
    #     num_actions=num_actions_example,
    #     hidden_dim_guidance_llm=hidden_dim_guidance,
    #     # ...
    # ).to(DEVICE)

    for round_num in range(num_rounds):
        print(f"--- Round {round_num + 1} ---")

        # --- Giai đoạn 1 (Biến thể): Tạo Mẫu Dương cho Flow Model ---
        # (Trong vòng đầu tiên, chúng ta dùng πg. Các vòng sau có thể dùng πg hoặc thậm chí φ đã cải thiện)
        positive_samples_for_flow = [] # list of (z0, z1, context_for_guidance, explanation_for_guidance)
        print("Generating positive samples for Flow Model...")
        # for context, actual_decision, all_actions in dataset:
            # explanation_initial = explanation_llm.generate_explanation(context) # Có thể không cần ở bước này
            # explanation_sentences = ["Placeholder explanation sentence 1.", "Placeholder for positive sample gen."] # Hoặc dùng giải thích "vàng" nếu có

            # prob_dist_from_guidance = guidance_llm.get_decision_distribution(context, explanation_sentences, all_actions)
            # actual_decision_idx = all_actions.index(actual_decision)

            # if torch.argmax(prob_dist_from_guidance) == actual_decision_idx: # Đây là mẫu dương
            #     z1 = prob_dist_from_guidance.detach().cpu()
            #     z0 = torch.randn_like(z1)
                # guidance_hidden_states = guidance_llm.get_last_layer_hidden_states(context, explanation_sentences, " ".join(all_actions))
                # positive_samples_for_flow.append((z0, z1, guidance_hidden_states.detach().cpu()))
        # print(f"Generated {len(positive_samples_for_flow)} positive samples.")

        # --- Giai đoạn 2: Huấn luyện Rectified Flow Model (φ) ---
        print("Training Rectified Flow Model (φ)...")
        # for epoch in range(NUM_EPOCHS_FLOW_MODEL):
            # total_flow_loss = 0
            # for z0_sample, z1_sample, guidance_hs_sample in positive_samples_for_flow: # Cần batching
            #     time_sample = torch.rand(1,1).to(DEVICE) # (1,1)
            #     loss = rectified_flow_model.train_step(
            #         z0_sample.unsqueeze(0).to(DEVICE),
            #         z1_sample.unsqueeze(0).to(DEVICE),
            #         time_sample,
            #         guidance_hs_sample.unsqueeze(0).to(DEVICE)
            #     )
            #     total_flow_loss += loss
            # print(f"Flow Model Epoch {epoch+1}, Avg Loss: {total_flow_loss / len(positive_samples_for_flow)}")

        # --- Giai đoạn 3: Huấn luyện EXPLANATION LLM (πε) bằng PPO ---
        print("Training EXPLANATION LLM (πε) with PPO...")
        # for epoch_expl in range(NUM_EPOCHS_EXPLANATION_LLM):
            # ppo_batch_contexts = []
            # ppo_batch_explanations_text = [] # Dạng text để PPO tokenizer xử lý
            # ppo_batch_rewards = []

            # for context, actual_decision, all_actions in dataset: # Cần batching
            #     explanation_sentences = explanation_llm.generate_explanation(context)
            #     per_sentence_rewards_for_ppo = []
            #     prev_p_hat_actual_decision = 0.0 # Hoặc giá trị từ phân phối đồng nhất

            #     cumulative_explanation = []
            #     for sent_idx, sentence in enumerate(explanation_sentences):
            #         cumulative_explanation.append(sentence)
                        # Lấy hidden states từ LLM Hướng dẫn cho ngữ cảnh và giải thích hiện tại
            #             current_guidance_hidden_states = guidance_llm.get_last_layer_hidden_states(
            #                 context, cumulative_explanation, " ".join(all_actions)
            #             ).to(DEVICE)

                        # Sử dụng φ để tạo phân phối quyết định ước tính p_hat
            #             z0_for_phi_inference = torch.randn(len(all_actions)).to(DEVICE)
            #             p_hat_distribution = rectified_flow_model.solve_ode_generate_distribution(
            #                 z0_for_phi_inference, current_guidance_hidden_states
            #             )

            #             actual_decision_idx = all_actions.index(actual_decision)
            #             p_hat_actual_decision = p_hat_distribution[actual_decision_idx].item()

            #             reward_for_sentence = p_hat_actual_decision - prev_p_hat_actual_decision
            #             per_sentence_rewards_for_ppo.append(reward_for_sentence)
            #             prev_p_hat_actual_decision = p_hat_actual_decision

                # Chuẩn bị dữ liệu cho PPO trainer của TRL
                # query_text = f"Context: {context}" # Hoặc prompt ban đầu của ExplanationLLM
                # response_text = " ".join(explanation_sentences)
                # ppo_batch_contexts.append(query_text)
                # ppo_batch_explanations_text.append(response_text)
                # ppo_batch_rewards.append(torch.tensor(per_sentence_rewards_for_ppo[-1])) # PPO của TRL thường nhận 1 reward cho cả response

            # Huấn luyện Explanation LLM bằng PPO (ví dụ với TRL)
            # stats = ppo_trainer.step(tokenized_queries, tokenized_responses, rewards_for_ppo_trainer)
            # print(f"Explanation LLM Epoch {epoch_expl+1}, PPO Mean Reward: {stats['ppo/returns/mean']}")
    pass

# --- Chạy chương trình ---
if __name__ == "__main__":
    # smac_data = load_dataset("smac")
    # main_training_loop(smac_data)
    print("Khung sườn mã cho Policy-to-Language. Cần triển khai chi tiết.")